# 동일한 함수 이름으로 다양한 함수를 처리하기 


   함수형 프로그램 처럼 동일한 함수이지만 함수의 매개변수가 차이가 있을 때 
   이를 동일한 함수를 이용해서 처리할 수 있는 구조를 만든다.
   
   multipleddispatch 모듈에서 dispatch를 데코레이터로 사용하면 각 타입에 따른 것을 등록해서 사용할 수 있다
   
       다중 디스패치 (일명 multimethods, 일반 함수 및 함수 오버로드)는 호출 인수에 따라 실행할 여러 함수 본문 중에서 선택합니다.

    

In [1]:
!pip install multipledispatch --upgrade

Requirement already up-to-date: multipledispatch in c:\programdata\anaconda3\lib\site-packages


In [60]:
from multipledispatch import dispatch

@dispatch(int, int)
def add(x, y):
    print(" int add call ")
    return x + y    

@dispatch(str, str)
def add(x, y):
    print(" string  add call ")
    return "%s + %s" % (x, y)

@dispatch(list, list)
def add(x, y):
    print(" list  add call ")
    return x + y

@dispatch(dict, dict)
def add(x, y):
    print(" dict  add call ")
    x.update(y)
    return x


In [61]:
print(add(5,5))
print(add("a","b"))

print(add([1,2,3],[4,5,6]))
print(add({"a": 10},{'b': [4,5,6]}))

 int add call 
10
 string  add call 
a + b
 list  add call 
[1, 2, 3, 4, 5, 6]
 dict  add call 
{'a': 10, 'b': [4, 5, 6]}


In [10]:
dir(dispatch)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [11]:
help(dispatch)

Help on function dispatch in module multipledispatch.core:

dispatch(*types, **kwargs)
    Dispatch function on the types of the inputs
    
    Supports dispatch on all non-keyword arguments.
    
    Collects implementations based on the function name.  Ignores namespaces.
    
    If ambiguous type signatures occur a warning is raised when the function is
    defined suggesting the additional method to break the ambiguity.
    
    Examples
    --------
    
    >>> @dispatch(int)
    ... def f(x):
    ...     return x + 1
    
    >>> @dispatch(float)
    ... def f(x):
    ...     return x - 1
    
    >>> f(3)
    4
    >>> f(3.0)
    2.0
    
    Specify an isolated namespace with the namespace keyword argument
    
    >>> my_namespace = dict()
    >>> @dispatch(int, namespace=my_namespace)
    ... def foo(x):
    ...     return x + 1
    
    Dispatch on instance methods within classes
    
    >>> class MyClass(object):
    ...     @dispatch(list)
    ...     def __init__(self

### 디스 패치를 만들면 하나의  클래스를 만들어 준다.

    실제함수는 funcs 내부에 보관되어 있다.
    

In [19]:
my_namespace = dict()
@dispatch(int, namespace=my_namespace)
def foo(x):
    return x + 1
@dispatch(float, namespace=my_namespace)
def foo(x):
    return x + 1

print(my_namespace)


{'foo': <dispatched foo>}


In [20]:
type(my_namespace['foo'])

multipledispatch.dispatcher.Dispatcher

In [18]:
dir(my_namespace['foo'])

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_cache',
 '_help',
 '_source',
 'add',
 'dispatch',
 'dispatch_iter',
 'doc',
 'funcs',
 'help',
 'name',
 'ordering',
 'register',
 'reorder',
 'resolve',
 'source']

In [15]:
my_namespace['foo'].funcs

{(int,): <function __main__.foo>, (float,): <function __main__.foo>}

In [55]:
nm = {}

class MyClass(object):
    
    @dispatch(object,int,namespace=nm)
    def __call__(cls,x):
        return " Class call "
    
    @dispatch(object,str, namespace=nm)
    def __call__(self, x):
        return " instace call " +x

In [59]:
my = MyClass()
print(my)
print(my(my,"str"))
print(my(MyClass,10))

 instace call str
 Class call 


In [56]:
nm['__call__'].funcs

{(object, int): <function __main__.MyClass.__call__>,
 (object, str): <function __main__.MyClass.__call__>}